In [1]:
import pandas as pd
import numpy as np
import os
from os.path import join
import re
import ast
import sys

In [2]:
RULES_PATH = '..//solvency2-rules//'
FILENAME_RULES = '2020-01-22 Set aanvullende controleregels Solvency II_tcm46-387021.xlsx'

## Construct test Solvency 2 instance (put here your own data)

In [3]:
RESULTS_PATH = '..\\results\\'
DATA_PATH = '..\\data\\'

df = pd.DataFrame()
df.index.name = "index"
files = [f for f in os.listdir(RESULTS_PATH) if os.path.isfile(os.path.join(RESULTS_PATH, f)) if f[-6:]=='pickle']
for file in files:
    new_df = pd.read_pickle(os.path.join(RESULTS_PATH, file))
    new_df.columns = [col.upper() for col in new_df.columns]
    if list(new_df.index) == [0]: # without z-axis for now
        for col in new_df.columns:
            if col not in df.columns:
                df[col] = new_df[col]
    else:
        for col in new_df.columns:
            if col not in df.columns:
                # we only pick the first line in the z-axis column
                df.loc[0, col] = new_df.iloc[0, new_df.columns.get_loc(col)]
        df.loc[0, new_df.index.name] = new_df.index[0]
df = df.astype(object)

In [4]:
df.head()

,"E.01.01.16.01,EC0020","E.01.01.16.01,EC0030","E.01.01.16.01,EC0040","E.01.01.16.01,EC0050","E.01.01.16.01,EC0060","E.01.01.16.01,EC0010","E.02.01.16.01,EC0010","E.02.01.16.01,ER0050","E.03.01.16.01,EC0020","E.03.01.16.01,ER0030",...,"SR.27.01.01.28,R2421,C0781","T.99.01.01.01,C0050","T.99.01.01.01,C0060","T.99.01.01.01,C0070","T.99.01.01.01,C0080","T.99.01.01.01,C0090","T.99.01.01.01,C0100","T.99.01.01.01,C0110","T.99.01.01.01,C0120","T.99.01.01.01,C0010"
index,,,,,,,,,,,,,,,,,,,,,
0,"IRAN, ISLAMIC REPUBLIC OF",s2c_CU:SCR,4.59446e+06,7.7809e+08,9.48777e+08,1,3.62888e+08,Pension entitlements,8.40734e+08,Home country,...,0,bpfy aq nmai jr,9.38006e+08,svvf vvzwwn,1999-10-10 00:00:00,80290,796320,0.6436,True,1


## Create simple taxonomy based on instance

In [5]:
df_taxo = pd.DataFrame(columns = ['datapoint', 'template', 'row', 'column', 'dtype'])
for idx, col in enumerate(df.columns):
    df_taxo.loc[idx, "datapoint"] = col.upper()
    df_taxo.loc[idx, "template"] = col[0:13].upper()
    df_taxo.loc[idx, "row"] = col[14:19].upper()
    df_taxo.loc[idx, "column"] = col[20:25].upper()
    df_taxo.loc[idx, "dtype"] = df.dtypes[idx]
df_taxo.head(5)

# for now we only use the list of templates in the instance
instance_templates = list(df_taxo.loc[:, 'template'].unique())
del df_taxo

## Read DNBs Additional Validation Rules

In [6]:
df_rules = pd.read_excel(os.path.join(RULES_PATH, FILENAME_RULES), header = 1)
df_rules = df_rules.set_index('ControleRegelCode')
df_rules = df_rules.drop('S.28.01_129', axis = 0) # double line, should be removed
df_rules = df_rules.drop('S.01.03_110', axis = 0) # double line, should be removed
df_rules.fillna("", inplace = True)


## Parse formulas

In [38]:
def replace_and_or(s):
    """Replace and by & and or by |, but not within strings"""
    if re.search(r"(.*?)\'(.*?)\'(.*)", s) is None: # input text does not contain strings
        s = s.replace("OR", "|")
        s = s.replace("AND", "&")
    for item in re.findall(r"(.*?)\'(.*?)\'(.*)", s):
        s = s.replace(item[0], item[0].replace("OR", "|"))
        s = s.replace(item[0], item[0].replace("AND", "&"))
        s = s.replace(item[2], replace_and_or(item[2]))
    return s

def replace_div_by_zero(s):
    """Replace division by adding a smal value to numerator"""
    item = re.search(r"{(.*?)}(/)({.*?})", s)
    if item is not None: # input text does not contain strings
        k = s.rfind("/")
        s = s[:k] + ".divide(" + s[k+1:].replace(item.group(3), item.group(3) + '.replace([0], -1))')
        print(s)
    return s

def preprocess(s):
    """Transform EVA2 code to Python Pandas code"""
    res = s
    res = res.replace("=" , "==")
    res = res.replace(">==" , ">=")
    res = res.replace("<==" , "<=")
    res = res.replace("<>", "!=")
    res = res.replace("< >", "!=") # the space between < and > should be deleted in EVA2
    res = res.replace('"', "'")
    res = replace_and_or(res)
    res = replace_div_by_zero(res)
    res = res.replace(" )", ")")
    res = res.replace(';', ",") # this should be corrected in EVA2
    return res

def datapoints2pandas(s):
    """Transform EVA2 datapoints to Python Pandas datapoints"""
    res = s
    datapoints = []
    for item in re.findall(r'{(.*?)}', res):
        res = res.replace("{"+item+"}", "df['"+item.upper()+"']")
        datapoints.append(item.upper())
    return res, datapoints

def add_brackets(s):
    """Add brackets around expressions with & and | (this is not consistent in EVA2)
    TODO: should not apply is AND or OR is in string text
    """
    item = re.search(r'(.*)([&|\|])(.*)', s) # & and | takes priority over other functions like ==
    if item is not None:
        return '('+ add_brackets(item.group(1)) + ') ' + item.group(2).strip() + ' (' + add_brackets(item.group(3)) + ')'
    else:
        item = re.search(r'(.*)([>|<|!=|<=|>=|==])(.*)', s)
        if item is not None:
            return add_brackets(item.group(1)) + item.group(2).strip() + add_brackets(item.group(3))
        else:
            return s.strip()
    
def expression2pandas(g):
    """Transform EVA2 conditional expression to Python Pandas code"""
    item = re.search(r'IF(.*)THEN(.*)', g)
    if item is not None:
        co_str = 'df[('+add_brackets(item.group(1))+') & ('+add_brackets(item.group(2))+")]"
        ex_str = 'df[('+add_brackets(item.group(1))+') & ~('+add_brackets(item.group(2))+")]"
    else:
        co_str = 'df[('+add_brackets(g)+')]'
        ex_str = 'df[~('+add_brackets(g)+')]'
    return co_str, ex_str

def evaluate_strings(df_data, co_str, ex_str):
    """Evaluate Python Pandas string for confirmation and exceptions"""
    try:
        co = len(eval(co_str, {'df': df_data, 'MAX': np.maximum, 'MIN': np.minimum, 'SUM': np.sum}))
        ex = len(eval(ex_str, {'df': df_data, 'MAX': np.maximum, 'MIN': np.minimum, 'SUM': np.sum}))
        return "Correctly parsed (#co=" + str(co)+", #ex="+str(ex)+")"
    except TypeError as e:
        return "Parse error: " + co_str + ": " + str(e)
    except:
        return "Parse error: " + co_str + ": UNKNOWN ERROR"   

In [39]:
def transform2pandas(df_data, rule_original):
    g = preprocess(rule_original)
    g, datapoints = datapoints2pandas(g)
    co_str, ex_str = expression2pandas(g)
    return co_str, ex_str, datapoints

def transform_rules(df_data, df_rules):
    for row in df_rules.index:
        rule_original = df_rules.loc[row, 'Formule']
        if not isinstance(rule_original, str):
            print("Rule " + row + ": " + "duplicate rule. ")
            rule_original = rule_original.values[0]
        else:
            co_str, ex_str, datapoints = transform2pandas(df_data, rule_original)
#             print(co_str, ex_str, datapoints)
            df_rules.loc[row, 'pandas co'] = co_str
            df_rules.loc[row, 'pandas ex'] = ex_str
            df_rules.at[row, 'datapoints'] = ''
            df_rules.at[row, 'datapoints'] = df_rules['datapoints'].astype('object')
            df_rules.at[row, 'datapoints'] = datapoints
            df_rules.at[row, 'templates'] = ''
            df_rules.at[row, 'templates'] = df_rules['templates'].astype('object')
            df_rules.at[row, 'templates'] = [datapoint[0:13].upper() for datapoint in datapoints]

def evaluate_rule(df, co_str, ex_str, datapoints, substitutions, expansion_dict):
    if datapoints == []:
        for item in substitutions.keys():
            co_str = co_str.replace(item, substitutions[item])
            ex_str = ex_str.replace(item, substitutions[item])
        print(evaluate_strings(df, co_str, ex_str))
    else:
        datapoint = datapoints.pop()
        if datapoint in expansion_dict.keys():
            for d in expansion_dict[datapoint]:
                substitutions[datapoint] = d
                evaluate_rule(df, co_str, ex_str, datapoints, substitutions, expansion_dict)
        else:
            evaluate_rule(df, co_str, ex_str, datapoints, substitutions, expansion_dict)

def evaluate_rules(df, df_rules):
    for idx in range(len(df_rules.index)):
        row = df_rules.index[idx]
        print(str(idx) + ": ", end='')
        rule_original = df_rules.loc[row, 'Formule']
        datapoints = df_rules.loc[row, 'datapoints']
        templates = df_rules.loc[row, 'templates']
        # are the templates in the rule in the instance?
        templates_not_found = []
        for template in templates:
            if template not in instance_templates:
                templates_not_found.append(template)
                
        if templates_not_found == []:
            datapoints_not_found = []
            expansion_dict = {}
            # are the datapoints in the rule in the instance?
            for datapoint in datapoints:
                if datapoint not in df.columns:
                    all_datapoints_found = False
                    new_list = []
                    if datapoint[14]=="C":
                        for col in df.columns:
                            reg = re.search(datapoint[0:14] + "R....," + datapoint[14:], col)
                            if reg:
                                new_list.append(reg.group(0))
                    if datapoint[14]=="R":
                        for col in df.columns:
                            reg = re.search(datapoint + ",C....", col)
                            if reg:
                                new_list.append(reg.group(0))
                    if new_list != []:
                        expansion_dict[datapoint] = new_list
                    else:
                        datapoints_not_found.append(datapoint)
            if datapoints_not_found == []:
                co_str = df_rules.loc[row, 'pandas co']
                ex_str = df_rules.loc[row, 'pandas ex']
                evaluate_rule(df, co_str, ex_str, datapoints, {}, expansion_dict)
            else:
                print("Datapoints not found: " +str(datapoints_not_found))
        else:
            print("Not all templates in instance: " + str(templates_not_found))

In [40]:
transform_rules(df, df_rules)

IF {S.27.01.01.04,c0220}!=0 THEN {S.27.01.01.04,c0240}=={S.27.01.01.04,c0230}.divide({S.27.01.01.04,c0220}.replace([0], -1))
IF {S.27.01.04.04,c0220}!=0 THEN {S.27.01.04.04,c0240}=={S.27.01.04.04,c0230}.divide({S.27.01.04.04,c0220}.replace([0], -1))
IF {S.27.01.01.05,c0310} >0.00 THEN {S.27.01.01.05,c0330}==({S.27.01.01.05,c0320}.divide({S.27.01.01.05,c0310}.replace([0], -1)))
IF {S.27.01.04.05,c0310} >0.00 THEN {S.27.01.04.05,c0330}==({S.27.01.04.05,c0320}.divide({S.27.01.04.05,c0310}.replace([0], -1)))
IF {S.27.01.01.06,r1950,c0420}  != 0 THEN {S.27.01.01.06,r1950,c0420}==({S.27.01.01.06,r1950,c0410}.divide({S.27.01.01.06,r1950,c0400}.replace([0], -1)))*100
IF {S.27.01.04.06,r1950,c0420} !=0 THEN {S.27.01.04.06,r1950,c0420}==({S.27.01.04.06,r1950,c0410}.divide({S.27.01.04.06,r1950,c0420}.replace([0], -1)))*100
IF {S.27.01.01.16,r2920,c0990}!=0 THEN {S.27.01.01.16,r2920,c1000}=={S.27.01.01.16,r2920,c1010}.divide({S.27.01.01.16,r2920,c0990}.replace([0], -1))
IF {S.27.01.04.16,r2920,c09

In [41]:
evaluate_rules(df, df_rules)

0: Not all templates in instance: ['S.01.01.01.01', 'S.01.01.01.01']
1: df[(df['S.01.01.02.01,R0580,C0010']=='REPORTED') & ~(df['S.01.01.02.01,R0590,C0010']!='REPORTED')]
Correctly parsed (#co=0, #ex=0)
2: df[~(df['S.01.01.02.01,R0590,C0010']!='REPORTED ')]
Correctly parsed (#co=1, #ex=0)
3: Not all templates in instance: ['S.02.01.01.01', 'S.02.01.01.01', 'S.02.01.01.01', 'S.01.01.01.01']
4: Not all templates in instance: ['S.02.01.01.01', 'S.02.01.01.01', 'S.02.01.01.01', 'S.01.01.04.01']
5: df[~(df['S.01.02.01.01,R0050,C0010']=='NETHERLANDS')]
Correctly parsed (#co=0, #ex=1)
6: Not all templates in instance: ['S.01.02.04.01']
7: df[~((df['S.01.02.01.01,R0070,C0010']=='DUTCH') | (df['S.01.02.01.01,R0070,C0010']=='ENGLISH'))]
Correctly parsed (#co=0, #ex=1)
8: Not all templates in instance: ['S.01.02.04.01', 'S.01.02.04.01']
9: df[~(df['S.01.02.01.01,R0100,C0010']=='REGULAR REPORTING')]
Correctly parsed (#co=0, #ex=1)
10: Not all templates in instance: ['S.01.02.04.01']
11: Not all te

46: df[(df['S.05.01.01.01,R0550,C0010']>0) & ~(df['S.05.01.01.01,R0700,C0010']>0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.05.01.01.01,R0550,C0020']>0) & ~(df['S.05.01.01.01,R0700,C0010']>0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.05.01.01.01,R0550,C0030']>0) & ~(df['S.05.01.01.01,R0700,C0010']>0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.05.01.01.01,R0550,C0040']>0) & ~(df['S.05.01.01.01,R0700,C0010']>0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.05.01.01.01,R0550,C0050']>0) & ~(df['S.05.01.01.01,R0700,C0010']>0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.05.01.01.01,R0550,C0060']>0) & ~(df['S.05.01.01.01,R0700,C0010']>0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.05.01.01.01,R0550,C0070']>0) & ~(df['S.05.01.01.01,R0700,C0010']>0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.05.01.01.01,R0550,C0080']>0) & ~(df['S.05.01.01.01,R0700,C0010']>0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.05.01.01.01,R0550,C0090']>0) & ~(df['S.05.01.01.01,R0700,C0010']>0)]
Correctly parsed (#co=1, #ex=

df[((df['S.05.01.02.01,R0110,C0200']!=0) | (df['S.05.01.02.01,R0310,C0200']!=0)) & ~(df['S.05.01.02.01,R0550,C0050']!=0)]
Correctly parsed (#co=1, #ex=0)
df[((df['S.05.01.02.01,R0110,C0200']!=0) | (df['S.05.01.02.01,R0310,C0200']!=0)) & ~(df['S.05.01.02.01,R0550,C0060']!=0)]
Correctly parsed (#co=1, #ex=0)
df[((df['S.05.01.02.01,R0110,C0200']!=0) | (df['S.05.01.02.01,R0310,C0200']!=0)) & ~(df['S.05.01.02.01,R0550,C0070']!=0)]
Correctly parsed (#co=1, #ex=0)
df[((df['S.05.01.02.01,R0110,C0200']!=0) | (df['S.05.01.02.01,R0310,C0200']!=0)) & ~(df['S.05.01.02.01,R0550,C0080']!=0)]
Correctly parsed (#co=1, #ex=0)
df[((df['S.05.01.02.01,R0110,C0200']!=0) | (df['S.05.01.02.01,R0310,C0200']!=0)) & ~(df['S.05.01.02.01,R0550,C0090']!=0)]
Correctly parsed (#co=1, #ex=0)
df[((df['S.05.01.02.01,R0110,C0200']!=0) | (df['S.05.01.02.01,R0310,C0200']!=0)) & ~(df['S.05.01.02.01,R0550,C0100']!=0)]
Correctly parsed (#co=1, #ex=0)
df[((df['S.05.01.02.01,R0110,C0200']!=0) | (df['S.05.01.02.01,R0310,C0200']!

111: df[(df['S.12.01.01.01,R0030,C0020']!=0) & ~(df['S.12.01.01.01,R0100,C0020']>=0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.12.01.01.01,R0030,C0030']!=0) & ~(df['S.12.01.01.01,R0100,C0020']>=0)]
Correctly parsed (#co=0, #ex=0)
df[(df['S.12.01.01.01,R0030,C0040']!=0) & ~(df['S.12.01.01.01,R0100,C0020']>=0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.12.01.01.01,R0030,C0050']!=0) & ~(df['S.12.01.01.01,R0100,C0020']>=0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.12.01.01.01,R0030,C0060']!=0) & ~(df['S.12.01.01.01,R0100,C0020']>=0)]
Correctly parsed (#co=0, #ex=0)
df[(df['S.12.01.01.01,R0030,C0070']!=0) & ~(df['S.12.01.01.01,R0100,C0020']>=0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.12.01.01.01,R0030,C0080']!=0) & ~(df['S.12.01.01.01,R0100,C0020']>=0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.12.01.01.01,R0030,C0090']!=0) & ~(df['S.12.01.01.01,R0100,C0020']>=0)]
Correctly parsed (#co=1, #ex=0)
df[(df['S.12.01.01.01,R0030,C0100']!=0) & ~(df['S.12.01.01.01,R0100,C0020']>=0)]
Correctly 

135: df[~(df['S.17.01.01.01,R0330,C0020']==df['S.17.01.01.01,R0050,C0020'] + df['S.17.01.01.01,R0140,C0020'] + df['S.17.01.01.01,R0240,C0020'])]
Correctly parsed (#co=0, #ex=1)
df[~(df['S.17.01.01.01,R0330,C0030']==df['S.17.01.01.01,R0050,C0020'] + df['S.17.01.01.01,R0140,C0020'] + df['S.17.01.01.01,R0240,C0020'])]
Correctly parsed (#co=0, #ex=1)
df[~(df['S.17.01.01.01,R0330,C0040']==df['S.17.01.01.01,R0050,C0020'] + df['S.17.01.01.01,R0140,C0020'] + df['S.17.01.01.01,R0240,C0020'])]
Correctly parsed (#co=0, #ex=1)
df[~(df['S.17.01.01.01,R0330,C0050']==df['S.17.01.01.01,R0050,C0020'] + df['S.17.01.01.01,R0140,C0020'] + df['S.17.01.01.01,R0240,C0020'])]
Correctly parsed (#co=0, #ex=1)
df[~(df['S.17.01.01.01,R0330,C0060']==df['S.17.01.01.01,R0050,C0020'] + df['S.17.01.01.01,R0140,C0020'] + df['S.17.01.01.01,R0240,C0020'])]
Correctly parsed (#co=0, #ex=1)
df[~(df['S.17.01.01.01,R0330,C0070']==df['S.17.01.01.01,R0050,C0020'] + df['S.17.01.01.01,R0140,C0020'] + df['S.17.01.01.01,R0240,C0020

...

df[(df['S.27.01.01.05,R1640,C0310']>0.00) & ~(df['S.27.01.01.05,R1940,C0330']==(df['S.27.01.01.05,R1940,C0320'].divide(df['S.27.01.01.05,R1640,C0310'].replace([0], -1))))]
Correctly parsed (#co=1, #ex=0)
df[(df['S.27.01.01.05,R1641,C0310']>0.00) & ~(df['S.27.01.01.05,R1940,C0330']==(df['S.27.01.01.05,R1940,C0320'].divide(df['S.27.01.01.05,R1641,C0310'].replace([0], -1))))]
Correctly parsed (#co=1, #ex=0)
df[(df['S.27.01.01.05,R1650,C0310']>0.00) & ~(df['S.27.01.01.05,R1940,C0330']==(df['S.27.01.01.05,R1940,C0320'].divide(df['S.27.01.01.05,R1650,C0310'].replace([0], -1))))]
Correctly parsed (#co=1, #ex=0)
df[(df['S.27.01.01.05,R1660,C0310']>0.00) & ~(df['S.27.01.01.05,R1940,C0330']==(df['S.27.01.01.05,R1940,C0320'].divide(df['S.27.01.01.05,R1660,C0310'].replace([0], -1))))]
Correctly parsed (#co=1, #ex=0)
df[(df['S.27.01.01.05,R1670,C0310']>0.00) & ~(df['S.27.01.01.05,R1940,C0330']==(df['S.27.01.01.05,R1940,C0320'].divide(df['S.27.01.01.05,R1670,C0310'].replace([0], -1))))]
Correctly pa

df[~(df['S.27.01.01.06,R1960,C0420']<=0)]
Correctly parsed (#co=1, #ex=0)
df[~(df['S.27.01.01.06,R1960,C0430']<=0)]
Correctly parsed (#co=0, #ex=1)
df[~(df['S.27.01.01.06,R1960,C0440']<=0)]
Correctly parsed (#co=1, #ex=0)
df[~(df['S.27.01.01.06,R1960,C0450']<=0)]
Correctly parsed (#co=1, #ex=0)
df[~(df['S.27.01.01.06,R1960,C0460']<=0)]
Correctly parsed (#co=0, #ex=1)
369: Not all templates in instance: ['S.27.01.04.06']
370: df[~(df['S.27.01.01.07,R2000,C0480']==2.5*df['S.27.01.01.07,R2000,C0470'])]
Correctly parsed (#co=0, #ex=1)
371: Not all templates in instance: ['S.27.01.04.07', 'S.27.01.04.07']
372: df[(df['S.27.01.01.08,R2100,C0540']!=0) & ~(df['S.27.01.01.08,R2100,C0540']==MAX(6000000, 50000*(df['S.27.01.01.08,R2100,C0520']+0.05*df['S.27.01.01.08,R2100,C0530']+0.95*MIN(df['S.27.01.01.08,R2100,C0530'], 20000))**0.5))]
Correctly parsed (#co=0, #ex=1)
373: Not all templates in instance: ['S.27.01.04.08', 'S.27.01.04.08', 'S.27.01.04.08', 'S.27.01.04.08', 'S.27.01.04.08']
374: df[~

412: df[~(df['S.27.01.01.15,R2810,C0960']<=0)]
Correctly parsed (#co=0, #ex=1)
df[~(df['S.27.01.01.15,R2810,C0970']<=0)]
Correctly parsed (#co=1, #ex=0)
df[~(df['S.27.01.01.15,R2810,C0980']<=0)]
Correctly parsed (#co=0, #ex=1)
413: Not all templates in instance: ['S.27.01.04.15']
414: df[(df['S.27.01.01.16,R2920,C0990']!=0) & ~(df['S.27.01.01.16,R2920,C1000']==df['S.27.01.01.16,R2920,C1010'].divide(df['S.27.01.01.16,R2920,C0990'].replace([0], -1)))]
Correctly parsed (#co=0, #ex=1)
415: Not all templates in instance: ['S.27.01.04.16', 'S.27.01.04.16', 'S.27.01.04.16', 'S.27.01.04.16']
416: df[~(df['S.27.01.01.16,R2900,C1010']==df['S.27.01.01.16,R2900,C0990']*df['S.27.01.01.16,R2900,C1000'])]
Correctly parsed (#co=0, #ex=1)
df[~(df['S.27.01.01.16,R2910,C1010']==df['S.27.01.01.16,R2900,C0990']*df['S.27.01.01.16,R2900,C1000'])]
Correctly parsed (#co=0, #ex=1)
df[~(df['S.27.01.01.16,R2920,C1010']==df['S.27.01.01.16,R2900,C0990']*df['S.27.01.01.16,R2900,C1000'])]
Correctly parsed (#co=0, #ex

469: df[((df['S.28.01.01.02,R0090,C0020']!=0) & ((df['S.05.01.01.01,R0110,C0080'] + df['S.05.01.01.01,R0120,C0080'])>0)) & ~(df['S.28.01.01.02,R0090,C0030']>0)]
Correctly parsed (#co=1, #ex=0)
470: df[((df['S.28.01.01.02,R0100,C0020']!=0) & ((df['S.05.01.01.01,R0110,C0090'] + df['S.05.01.01.01,R0120,C0090'])>0)) & ~(df['S.28.01.01.02,R0100,C0030']>0)]
Correctly parsed (#co=1, #ex=0)
471: df[((df['S.28.01.01.02,R0110,C0020']!=0) & ((df['S.05.01.01.01,R0110,C0100'] + df['S.05.01.01.01,R0120,C0100'])>0)) & ~(df['S.28.01.01.02,R0110,C0030']>0)]
Correctly parsed (#co=1, #ex=0)
472: df[((df['S.28.01.01.02,R0120,C0020']!=0) & ((df['S.05.01.01.01,R0110,C0110'] + df['S.05.01.01.01,R0120,C0110'])>0)) & ~(df['S.28.01.01.02,R0120,C0030']>0)]
Correctly parsed (#co=1, #ex=0)
473: df[((df['S.28.01.01.02,R0130,C0020']!=0) & ((df['S.05.01.01.01,R0110,C0120'] + df['S.05.01.01.01,R0120,C0120'])>0)) & ~(df['S.28.01.01.02,R0130,C0030']>0)]
Correctly parsed (#co=1, #ex=0)
474: df[((df['S.28.01.01.02,R0140,C

554: Not all templates in instance: ['S.32.01.04.01', 'S.32.01.04.01']
555: Not all templates in instance: ['S.32.01.04.01', 'S.32.01.04.01']
556: Not all templates in instance: ['S.32.01.04.01', 'S.32.01.04.01', 'S.32.01.04.01', 'S.32.01.04.01', 'S.32.01.04.01']
557: Not all templates in instance: ['S.32.01.04.01', 'S.32.01.04.01', 'S.32.01.04.01', 'S.32.01.04.01', 'S.32.01.04.01', 'S.32.01.04.01', 'S.32.01.04.01']
558: Not all templates in instance: ['S.32.01.04.01', 'S.32.01.04.01', 'S.32.01.04.01', 'S.32.01.04.01', 'S.32.01.04.01', 'S.32.01.04.01']
559: Not all templates in instance: ['S.32.01.04.01', 'S.32.01.04.01']
560: Not all templates in instance: ['S.32.01.04.01', 'S.32.01.04.01', 'S.01.02.04.01', 'S.32.01.04.01']
561: Not all templates in instance: ['S.32.01.04.01', 'S.32.01.04.01', 'S.01.02.04.01', 'S.32.01.04.01']
562: Not all templates in instance: ['S.32.01.04.01', 'S.32.01.04.01']
563: Not all templates in instance: ['S.32.01.04.01', 'S.32.01.04.01']
564: Not all templ

In [34]:
df['S.27.01.01.04,R1620,C0230'].divide(df['S.27.01.01.04,R1370,C0220']).replace(0, -232)

index
0   -232
dtype: int64